In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler


In [2]:
df = pd.read_csv('./Divar_filled.csv')

C:\Users\Asus\AppData\Local\Temp\ipykernel_30868\3949448737.py:1: DtypeWarning: Columns (11,27,29,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./Divar_filled.csv')


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 62 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   Unnamed: 0                  1000000 non-null  int64  
 1   cat2_slug                   1000000 non-null  object 
 2   cat3_slug                   999999 non-null   object 
 3   city_slug                   999998 non-null   object 
 4   neighborhood_slug           437139 non-null   object 
 5   created_at_month            1000000 non-null  object 
 6   user_type                   1000000 non-null  object 
 7   description                 1000000 non-null  object 
 8   title                       999946 non-null   object 
 9   rent_mode                   1000000 non-null  object 
 10  rent_value                  351322 non-null   float64
 11  rent_to_single              19 non-null       object 
 12  rent_type                   1000000 non-null  object 
 13

In [4]:
numeric_features = [
    'price_value', 'building_size', 'land_size', 'rooms_count',
    'floor', 'total_floors_count', 'unit_per_floor', 'construction_year',
    'location_latitude', 'location_longitude'
]

categorical_features = [
    'city_slug', 'neighborhood_slug', 'deed_type',
    'cat2_slug', 'cat3_slug', 'user_type', 'building_direction', 'floor_material'
]

boolean_features = [
    'has_balcony', 'has_elevator', 'has_warehouse', 'has_parking',
    'is_rebuilt', 'has_water', 'has_warm_water_provider', 'has_electricity',
    'has_gas', 'has_heating_system', 'has_cooling_system', 'has_restroom',
    'has_security_guard', 'has_barbecue', 'has_pool', 'has_jacuzzi', 'has_sauna'
]

In [5]:
df2 = pd.DataFrame()
for _ in df.columns:
    if df[_].notna().sum() >300000:
        df2[_] = df[_]


df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 45 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   Unnamed: 0               1000000 non-null  int64  
 1   cat2_slug                1000000 non-null  object 
 2   cat3_slug                999999 non-null   object 
 3   city_slug                999998 non-null   object 
 4   neighborhood_slug        437139 non-null   object 
 5   created_at_month         1000000 non-null  object 
 6   user_type                1000000 non-null  object 
 7   description              1000000 non-null  object 
 8   title                    999946 non-null   object 
 9   rent_mode                1000000 non-null  object 
 10  rent_value               351322 non-null   float64
 11  rent_type                1000000 non-null  object 
 12  price_mode               1000000 non-null  object 
 13  price_value              568346 non-null   

In [6]:
df = df2.copy()

## Filling neighbourhood

In [7]:
cols = ['location_latitude', 'location_longitude', 'location_radius', 'neighborhood_slug']
df_subset = df[cols]

In [8]:
known = df_subset[df_subset['neighborhood_slug'].notna()]
unknown = df_subset[df_subset['neighborhood_slug'].isna()]

In [9]:
known = known.dropna(subset=['location_latitude', 'location_longitude', 'location_radius'])


In [10]:
counts = known['neighborhood_slug'].value_counts()
valid_neighborhoods = counts[counts >= 2].index
known = known[known['neighborhood_slug'].isin(valid_neighborhoods)]


In [11]:
le = LabelEncoder()
known['neighborhood_encoded'] = le.fit_transform(known['neighborhood_slug'])

In [12]:
X = known[['location_latitude', 'location_longitude', 'location_radius']]
y = known['neighborhood_encoded']


In [13]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [14]:
k = 5  
knn = KNeighborsClassifier(n_neighbors=k, weights='distance')
knn.fit(X_train, y_train)

KNeighborsClassifier(weights='distance')

In [15]:
y_pred_val = knn.predict(X_val)
accuracy = accuracy_score(y_val, y_pred_val)
print(f"KNN Accuracy for neighborhood prediction: {accuracy:.2%}")


KNN Accuracy for neighborhood prediction: 85.20%


In [16]:
X_test = unknown[['location_latitude', 'location_longitude', 'location_radius']].dropna()


In [17]:
y_pred_test = knn.predict(X_test)
predicted_labels = le.inverse_transform(y_pred_test)

In [18]:
df.loc[X_test.index, 'neighborhood_slug'] = predicted_labels

print(f"Filled {len(X_test)} missing neighborhood_slug values.")

Filled 117692 missing neighborhood_slug values.


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 45 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   Unnamed: 0               1000000 non-null  int64  
 1   cat2_slug                1000000 non-null  object 
 2   cat3_slug                999999 non-null   object 
 3   city_slug                999998 non-null   object 
 4   neighborhood_slug        554831 non-null   object 
 5   created_at_month         1000000 non-null  object 
 6   user_type                1000000 non-null  object 
 7   description              1000000 non-null  object 
 8   title                    999946 non-null   object 
 9   rent_mode                1000000 non-null  object 
 10  rent_value               351322 non-null   float64
 11  rent_type                1000000 non-null  object 
 12  price_mode               1000000 non-null  object 
 13  price_value              568346 non-null   

## Drop Unnecessary Columns

In [20]:
df['property_type'].value_counts()

property_type
jungle             950378
other               42797
beach                4188
jungle-mountain      2399
mountain              238
Name: count, dtype: int64

In [21]:
df.drop(['location_radius','location_longitude','location_latitude','property_type','floor_material','has_cooling_system','has_heating_system','has_warm_water_provider','room_count','transformable_rent','transformable_credit','transformable_price','rent_credit_transform','title','description','created_at_month','Unnamed: 0'] , axis=1 , inplace = True)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 28 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   cat2_slug           1000000 non-null  object 
 1   cat3_slug           999999 non-null   object 
 2   city_slug           999998 non-null   object 
 3   neighborhood_slug   554831 non-null   object 
 4   user_type           1000000 non-null  object 
 5   rent_mode           1000000 non-null  object 
 6   rent_value          351322 non-null   float64
 7   rent_type           1000000 non-null  object 
 8   price_mode          1000000 non-null  object 
 9   price_value         568346 non-null   float64
 10  credit_mode         1000000 non-null  object 
 11  credit_value        352095 non-null   float64
 12  building_size       980394 non-null   float64
 13  floor               541748 non-null   object 
 14  rooms_count         1000000 non-null  object 
 15  total_floors_cou

In [23]:
df = df.dropna(subset=["neighborhood_slug"])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 554831 entries, 0 to 999999
Data columns (total 28 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   cat2_slug           554831 non-null  object 
 1   cat3_slug           554830 non-null  object 
 2   city_slug           554831 non-null  object 
 3   neighborhood_slug   554831 non-null  object 
 4   user_type           554831 non-null  object 
 5   rent_mode           554831 non-null  object 
 6   rent_value          212970 non-null  float64
 7   rent_type           554831 non-null  object 
 8   price_mode          554831 non-null  object 
 9   price_value         308760 non-null  float64
 10  credit_mode         554831 non-null  object 
 11  credit_value        213427 non-null  float64
 12  building_size       544945 non-null  float64
 13  floor               367562 non-null  object 
 14  rooms_count         554831 non-null  object 
 15  total_floors_count  214792 non-null  ob

In [24]:
df.rent_type.unique()

array(['rent_credit', 'full_credit'], dtype=object)

In [25]:
df[["rent_value", "rent_type", "credit_value", "credit_mode", "price_value", "price_mode"]].sample(10)


,rent_value,rent_type,credit_value,credit_mode,price_value,price_mode
602767,NaN,rent_credit,NaN,مقطوع,2.000000e+09,مقطوع
178618,1.500000e+09,rent_credit,5.000000e+09,مقطوع,NaN,مقطوع
39669,NaN,rent_credit,NaN,مقطوع,4.100000e+10,مقطوع
245903,NaN,rent_credit,NaN,مقطوع,1.111111e+09,مقطوع
32830,NaN,rent_credit,NaN,مقطوع,5.300000e+09,مقطوع
843173,NaN,rent_credit,NaN,مقطوع,1.925000e+09,مقطوع
61774,NaN,rent_credit,NaN,مقطوع,3.000000e+09,مقطوع
318053,NaN,rent_credit,NaN,مقطوع,2.750000e+09,مقطوع
157993,NaN,rent_credit,NaN,مقطوع,1.910000e+09,مقطوع
687934,NaN,rent_credit,NaN,مقطوع,8.550000e+09,مقطوع


In [1]:
df = df.reset_index(names="id")

long_values = df.melt(
    id_vars=["id"],
    value_vars=["rent_value", "credit_value", "price_value"],
    var_name="flag",
    value_name="value"
)

long_modes = df.melt(
    id_vars=["id"],
    value_vars=["rent_type", "credit_mode", "price_mode"],
    var_name="flag_mode",
    value_name="mode"
)

long_values["flag"] = long_values["flag"].str.replace("_value", "", regex=False)
long_modes["flag_mode"] = long_modes["flag_mode"].str.replace("_type", "", regex=False)
long_modes["flag_mode"] = long_modes["flag_mode"].str.replace("_mode", "", regex=False)

long_df = long_values.merge(
    long_modes, left_on=["id", "flag"], right_on=["id", "flag_mode"], how="left"
).drop(columns="flag_mode")

long_df = long_df.dropna(subset=["value"])

df_with_flag = df.merge(long_df, on="id", how="left")

NameError: name 'df' is not defined

In [27]:
df_with_flag

,id,cat2_slug,cat3_slug,city_slug,neighborhood_slug,user_type,rent_mode,rent_value,rent_type,price_mode,...,construction_year,is_rebuilt,has_restroom,building_direction,has_pool,has_jacuzzi,has_sauna,flag,value,mode
0,0,temporary-rent,villa,karaj,mehrshahr,مشاور املاک,مقطوع,NaN,rent_credit,مقطوع,...,NaN,NaN,NaN,NaN,False,False,False,NaN,NaN,NaN
1,1,residential-sell,apartment-sell,tehran,gholhak,مشاور املاک,مقطوع,NaN,rent_credit,مقطوع,...,۱۳۸۴,NaN,NaN,NaN,False,False,False,price,8.500000e+09,مقطوع
2,2,residential-rent,apartment-rent,tehran,tohid,شخصی,مقطوع,26000000.0,rent_credit,مقطوع,...,۱۴۰۱,False,NaN,NaN,False,False,False,rent,2.600000e+07,rent_credit
3,2,residential-rent,apartment-rent,tehran,tohid,شخصی,مقطوع,26000000.0,rent_credit,مقطوع,...,۱۴۰۱,False,NaN,NaN,False,False,False,credit,7.500000e+08,مقطوع
4,3,commercial-rent,office-rent,tehran,elahiyeh,مشاور املاک,مقطوع,95000000.0,rent_credit,مقطوع,...,۱۴۰۰,NaN,NaN,NaN,False,False,False,rent,9.500000e+07,rent_credit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
767722,999995,residential-sell,apartment-sell,kermanshah,khorramdasht,مشاور املاک,مقطوع,NaN,rent_credit,مقطوع,...,۱۴۰۳,NaN,NaN,NaN,False,False,False,price,7.470000e+09,مقطوع
767723,999996,residential-rent,apartment-rent,tehran,darya,مشاور املاک,مقطوع,45000000.0,rent_credit,مقطوع,...,۱۴۰۳,NaN,NaN,NaN,False,False,False,rent,4.500000e+07,rent_credit
767724,999996,residential-rent,apartment-rent,tehran,darya,مشاور املاک,مقطوع,45000000.0,rent_credit,مقطوع,...,۱۴۰۳,NaN,NaN,NaN,False,False,False,credit,1.000000e+09,مقطوع
767725,999999,residential-rent,apartment-rent,tehran,jeyhoun,مشاور املاک,مجانی,0.0,rent_credit,مقطوع,...,۱۳۸۲,True,squat,NaN,False,False,False,rent,0.000000e+00,rent_credit


In [28]:
df = df_with_flag.drop(['id',"rent_value", "credit_value", "price_value","rent_type", "credit_mode", "price_mode",'rent_mode','mode'] , axis = 1 )

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 767727 entries, 0 to 767726
Data columns (total 23 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   cat2_slug           767727 non-null  object 
 1   cat3_slug           767726 non-null  object 
 2   city_slug           767727 non-null  object 
 3   neighborhood_slug   767727 non-null  object 
 4   user_type           767727 non-null  object 
 5   building_size       757840 non-null  float64
 6   floor               526434 non-null  object 
 7   rooms_count         767727 non-null  object 
 8   total_floors_count  297891 non-null  object 
 9   unit_per_floor      295946 non-null  object 
 10  has_balcony         413302 non-null  object 
 11  has_elevator        767727 non-null  bool   
 12  has_warehouse       767727 non-null  bool   
 13  has_parking         767727 non-null  bool   
 14  construction_year   699012 non-null  object 
 15  is_rebuilt          411102 non-nul

## Data Engineering

In [30]:
df['construction_year'].value_counts()

construction_year
۱۴۰۳           86836
۱۳۹۰           54171
۱۳۹۵           44839
۱۴۰۰           44702
۱۴۰۲           44013
۱۳۸۵           34109
۱۳۹۸           33155
۱۳۹۷           31333
۱۳۹۶           29725
۱۴۰۱           29067
۱۳۹۳           25295
۱۳۹۹           25179
۱۳۸۸           23990
۱۳۸۰           22794
۱۳۹۲           21978
۱۳۹۴           21824
قبل از ۱۳۷۰    16165
۱۳۸۹           15802
۱۳۹۱           14040
۱۳۸۷           13821
۱۳۸۶           12874
۱۳۸۳           10479
۱۳۸۴            8315
۱۳۸۲            6870
۱۳۷۵            6568
۱۳۷۱            4831
۱۳۸۱            3352
۱۳۷۸            2858
۱۳۷۹            2247
۱۳۷۷            1911
۱۳۷۳            1700
۱۳۷۲            1689
۱۳۷۶            1396
۱۳۷۴            1084
Name: count, dtype: int64

In [31]:
persian_to_english_map = str.maketrans('۰۱۲۳۴۵۶۷۸۹', '0123456789')

df['construction_year'] = (
    df['construction_year']
    .astype(str)
    .apply(lambda x: x.translate(persian_to_english_map) if isinstance(x, str) else x)
    .str.extract(r'(\d+)', expand=False)   
)


df['age'] = 1404 -  pd.to_numeric(df['construction_year'])
df.drop(['construction_year'] , axis = 1 , inplace = True)

print(df['age'].sort_values().unique())

[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. nan]


In [32]:
df.columns
for _ in df.columns:
    df[_]=df[_].replace("unselect", np.nan)
    print(_ ,df[_].unique())

cat2_slug ['temporary-rent' 'residential-sell' 'residential-rent' 'commercial-rent'
 'commercial-sell' 'real-estate-services']
cat3_slug ['villa' 'apartment-sell' 'apartment-rent' 'office-rent' 'plot-old'
 'house-villa-sell' 'house-villa-rent' 'shop-sell' 'shop-rent'
 'office-sell' 'presell' 'industry-agriculture-business-sell'
 'industry-agriculture-business-rent' 'suite-apartment' 'partnership'
 'workspace' nan]
city_slug ['karaj' 'tehran' 'mashhad' 'ahvaz' 'mahdasht-city' 'rasht' 'shiraz'
 'gonabad' 'andisheh-new-town' 'chamestan' 'urmia' 'nur' 'amol' 'isfahan'
 'ilam' 'pakdasht-city' 'chalus' 'eslamshahr' 'lavasan-city' 'mohammadieh'
 'izadshahr' 'nowshahr' 'neyshabur' 'rudehen' 'torbat-jam' 'polsefid'
 'kerman' 'khorramabad' 'bandar-abbas' 'ramiyan' 'sanandaj' 'qom'
 'qarchak-city' 'garmdareh' 'kish' 'kiasar' 'pardis-city' 'birjand'
 'ardabil' 'golestan-baharestan' 'kahrizak' 'rudsar'
 'safadasht-industrial-city' 'varamin-city' 'qods-city' 'hamedan'
 'parand-city' 'robat-karim' 'n

In [33]:
df['rooms_count'] = df['rooms_count'].replace({
    'پنج یا بیشتر': 5,
    'چهار': 4,
    'سه': 3,
    'یک': 1,
    'دو': 2,
    'بدون اتاق': 0
})
df['rooms_count'].unique()

C:\Users\Asus\AppData\Local\Temp\ipykernel_30868\1842464251.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['rooms_count'] = df['rooms_count'].replace({


array([3, 1, 2, 0, 4, 5])

In [34]:
df['unit_per_floor'] = df['unit_per_floor'].replace({'more_than_8': 9})
df['unit_per_floor'].unique()

array([nan, '2', '4', '1', '3', '8', '6', '5', '7', 9], dtype=object)

In [35]:
df['floor'] = df['floor'].replace({'30+': 30})
df['floor'].unique()

array([nan, '3', '4', '1', '2', '5', '0', '18', '8', '6', '12', '7', '11',
       '13', '9', '10', '19', '16', '-1', '21', '15', '14', '17', '20',
       '22', '23', '25', '24', 30, '30', '26', '28', '29', '27', 3.0, 2.0,
       1.0, 0.0, 4.0, 20.0, 7.0, 5.0, 18.0, 8.0, 6.0, 22.0, 10.0, 14.0,
       9.0], dtype=object)

In [36]:
df['total_floors_count'] = df['total_floors_count'].replace({'30+': 30})
df['total_floors_count'].unique()

array([nan, '6', '3', '5', '4', '2', '7', '8', '12', '11', '17', '9',
       '10', '25', '18', '19', '16', '21', '20', '15', '22', '13', '14',
       '24', '29', '26', '30', 30, '27', '23', '28', 4.0, 5.0, 2.0, 3.0,
       25.0, 6.0, 7.0, 20.0, 10.0, 8.0, 11.0, 21.0, 19.0, 13.0],
      dtype=object)

## Numericals to float

In [37]:
Numerical = ['floor','rooms_count','total_floors_count','unit_per_floor']

In [38]:
df3 = df.copy()

In [39]:
for i in Numerical:
    df[i] = df[i].astype(float)

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 767727 entries, 0 to 767726
Data columns (total 23 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   cat2_slug           767727 non-null  object 
 1   cat3_slug           767726 non-null  object 
 2   city_slug           767727 non-null  object 
 3   neighborhood_slug   767727 non-null  object 
 4   user_type           767727 non-null  object 
 5   building_size       757840 non-null  float64
 6   floor               526434 non-null  float64
 7   rooms_count         767727 non-null  float64
 8   total_floors_count  297822 non-null  float64
 9   unit_per_floor      295780 non-null  float64
 10  has_balcony         413132 non-null  object 
 11  has_elevator        767727 non-null  bool   
 12  has_warehouse       767727 non-null  bool   
 13  has_parking         767727 non-null  bool   
 14  is_rebuilt          411102 non-null  object 
 15  has_restroom        359019 non-nul

In [41]:
df.columns
for _ in df.columns:
    print(_ ,df[_].unique())

cat2_slug ['temporary-rent' 'residential-sell' 'residential-rent' 'commercial-rent'
 'commercial-sell' 'real-estate-services']
cat3_slug ['villa' 'apartment-sell' 'apartment-rent' 'office-rent' 'plot-old'
 'house-villa-sell' 'house-villa-rent' 'shop-sell' 'shop-rent'
 'office-sell' 'presell' 'industry-agriculture-business-sell'
 'industry-agriculture-business-rent' 'suite-apartment' 'partnership'
 'workspace' nan]
city_slug ['karaj' 'tehran' 'mashhad' 'ahvaz' 'mahdasht-city' 'rasht' 'shiraz'
 'gonabad' 'andisheh-new-town' 'chamestan' 'urmia' 'nur' 'amol' 'isfahan'
 'ilam' 'pakdasht-city' 'chalus' 'eslamshahr' 'lavasan-city' 'mohammadieh'
 'izadshahr' 'nowshahr' 'neyshabur' 'rudehen' 'torbat-jam' 'polsefid'
 'kerman' 'khorramabad' 'bandar-abbas' 'ramiyan' 'sanandaj' 'qom'
 'qarchak-city' 'garmdareh' 'kish' 'kiasar' 'pardis-city' 'birjand'
 'ardabil' 'golestan-baharestan' 'kahrizak' 'rudsar'
 'safadasht-industrial-city' 'varamin-city' 'qods-city' 'hamedan'
 'parand-city' 'robat-karim' 'n

## Drop Records with nan

In [42]:
df = df.dropna()

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 171996 entries, 10 to 767721
Data columns (total 23 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   cat2_slug           171996 non-null  object 
 1   cat3_slug           171996 non-null  object 
 2   city_slug           171996 non-null  object 
 3   neighborhood_slug   171996 non-null  object 
 4   user_type           171996 non-null  object 
 5   building_size       171996 non-null  float64
 6   floor               171996 non-null  float64
 7   rooms_count         171996 non-null  float64
 8   total_floors_count  171996 non-null  float64
 9   unit_per_floor      171996 non-null  float64
 10  has_balcony         171996 non-null  object 
 11  has_elevator        171996 non-null  bool   
 12  has_warehouse       171996 non-null  bool   
 13  has_parking         171996 non-null  bool   
 14  is_rebuilt          171996 non-null  object 
 15  has_restroom        171996 non-null  o

In [44]:
df['has_balcony'] = df['has_balcony'].map({'false': False, 'true': True})
df['has_balcony'].value_counts().sort_values()

has_balcony
False     22453
True     140905
Name: count, dtype: int64

In [45]:
df['has_balcony'] = df['has_balcony'].astype(bool)

In [46]:
df['is_rebuilt'] = df['is_rebuilt'].astype(bool)

In [47]:
df.columns
for _ in df.columns:
    print(_ ,df[_].unique())

cat2_slug ['residential-sell' 'residential-rent']
cat3_slug ['apartment-sell' 'apartment-rent']
city_slug ['mahdasht-city' 'mashhad' 'urmia' 'shiraz' 'tehran' 'isfahan'
 'eslamshahr' 'karaj' 'andisheh-new-town' 'ahvaz' 'khorramabad' 'rudsar'
 'robat-karim' 'rasht' 'varamin-city' 'qom' 'fardis-city' 'maragheh'
 'pardis-city' 'nasimshahr' 'kermanshah' 'tabriz' 'vahidieh' 'arak'
 'someh-sara' 'sadra' 'gorgan' 'hashtgerd-city' 'mohammad-shahr'
 'qods-city' 'langarud' 'shirvan' 'saveh' 'bumehen' 'kerman' 'garmdareh'
 'golbahar-city' 'bandar-abbas' 'kish' 'parand-city' 'birjand' 'sari'
 'hamedan' 'bandar-anzali' 'neyshabur' 'shahriar-city' 'torbat-jam'
 'lahijan' 'qarchak-city' 'ardabil' 'golpayegan' 'nowshahr' 'sabashahr'
 'maku' 'pakdasht-city' 'kashan' 'qorveh' 'gonbad-kavus'
 'golestan-baharestan' 'borujerd' 'sanandaj' 'talesh' 'qazvin'
 'minoodasht' 'charbagh-alborz' 'abadan' 'sahand'
 'safadasht-industrial-city' 'amlash' 'royan' 'astaneh-ashrafiyeh'
 'nazarabad' 'babolsar' 'javanrud' '

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 171996 entries, 10 to 767721
Data columns (total 23 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   cat2_slug           171996 non-null  object 
 1   cat3_slug           171996 non-null  object 
 2   city_slug           171996 non-null  object 
 3   neighborhood_slug   171996 non-null  object 
 4   user_type           171996 non-null  object 
 5   building_size       171996 non-null  float64
 6   floor               171996 non-null  float64
 7   rooms_count         171996 non-null  float64
 8   total_floors_count  171996 non-null  float64
 9   unit_per_floor      171996 non-null  float64
 10  has_balcony         171996 non-null  bool   
 11  has_elevator        171996 non-null  bool   
 12  has_warehouse       171996 non-null  bool   
 13  has_parking         171996 non-null  bool   
 14  is_rebuilt          171996 non-null  bool   
 15  has_restroom        171996 non-null  o

## Ordinal Encoding

In [49]:
cat_cols = ['flag', 'has_restroom','is_rebuilt','user_type','neighborhood_slug','city_slug','cat3_slug','cat2_slug','building_direction']

In [50]:
encoder = OrdinalEncoder()

In [51]:
df[cat_cols] = encoder.fit_transform(df[cat_cols])

In [52]:
mappings = {}
for i, col in enumerate(cat_cols):
    mappings[col] = dict(zip(encoder.categories_[i], range(len(encoder.categories_[i]))))

In [53]:
mappings

{'flag': {'credit': 0, 'price': 1, 'rent': 2},
 'has_restroom': {'seat': 0, 'squat': 1, 'squat_seat': 2},
 'is_rebuilt': {np.False_: 0, np.True_: 1},
 'user_type': {'شخصی': 0, 'مشاور املاک': 1},
 'neighborhood_slug': {'10-day': 0,
  '15khordad': 1,
  '17shahrivar': 2,
  '24-metri': 3,
  '350-metri': 4,
  'Baraghan': 5,
  'Kuy-e-Seyyedi': 6,
  'Salarieh': 7,
  'Shahrak-e-Shirin': 8,
  'Shahrak-e-Taleqani': 9,
  'abbas-abad': 10,
  'abbas-abad-mashhad': 11,
  'abbasabad': 12,
  'abdol-abad': 13,
  'abdollah-abad': 14,
  'abdolmotalleb': 15,
  'abiverdi': 16,
  'abjavar': 17,
  'abkuh': 18,
  'aboozar': 19,
  'aboutaleb': 20,
  'abouzar': 21,
  'abouzar-mashhad': 22,
  'abr': 23,
  'abrisham': 24,
  'abshar': 25,
  'abshar-isfahan': 26,
  'abshar-tehran': 27,
  'abunasr': 28,
  'abyari': 29,
  'adlekhomeinisquare': 30,
  'afaran': 31,
  'afifabad': 32,
  'afsariyeh': 33,
  'aghdasieh': 34,
  'ahadabad': 35,
  'ahang': 36,
  'ahmad-abad': 37,
  'ahmad-abad-isfahan': 38,
  'ahmadabad': 39,


In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 171996 entries, 10 to 767721
Data columns (total 23 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   cat2_slug           171996 non-null  float64
 1   cat3_slug           171996 non-null  float64
 2   city_slug           171996 non-null  float64
 3   neighborhood_slug   171996 non-null  float64
 4   user_type           171996 non-null  float64
 5   building_size       171996 non-null  float64
 6   floor               171996 non-null  float64
 7   rooms_count         171996 non-null  float64
 8   total_floors_count  171996 non-null  float64
 9   unit_per_floor      171996 non-null  float64
 10  has_balcony         171996 non-null  bool   
 11  has_elevator        171996 non-null  bool   
 12  has_warehouse       171996 non-null  bool   
 13  has_parking         171996 non-null  bool   
 14  is_rebuilt          171996 non-null  float64
 15  has_restroom        171996 non-null  f

In [55]:
df["neighborhood_encoded"] = df.groupby("neighborhood_slug")["value"].transform("mean")


In [56]:
df.drop(['neighborhood_slug'] , axis = 1 , inplace= True)

### handling Outliers

In [57]:
num_cols = ["building_size", "floor", "total_floors_count", "unit_per_floor", "value", "neighborhood_encoded"]

df_num = df[num_cols].copy()

In [58]:
for col in num_cols:
    Q1 = df_num[col].quantile(0.25)
    Q3 = df_num[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    df_num[col] = np.clip(df_num[col], lower, upper)


### Log transform

In [59]:
min_vals = df_num.min()
if (min_vals < 0).any():
    shift = abs(min_vals.min()) + 1  # shift so minimum = 1
    df_num = df_num + shift

In [60]:
df_num = np.log1p(df_num)

In [61]:
df_num = df_num.replace([np.inf, -np.inf], np.nan).fillna(0)

### Scaling

In [62]:
scaler = MinMaxScaler()
df[num_cols] = scaler.fit_transform(df_num)

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 171996 entries, 10 to 767721
Data columns (total 23 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   cat2_slug             171996 non-null  float64
 1   cat3_slug             171996 non-null  float64
 2   city_slug             171996 non-null  float64
 3   user_type             171996 non-null  float64
 4   building_size         171996 non-null  float64
 5   floor                 171996 non-null  float64
 6   rooms_count           171996 non-null  float64
 7   total_floors_count    171996 non-null  float64
 8   unit_per_floor        171996 non-null  float64
 9   has_balcony           171996 non-null  bool   
 10  has_elevator          171996 non-null  bool   
 11  has_warehouse         171996 non-null  bool   
 12  has_parking           171996 non-null  bool   
 13  is_rebuilt            171996 non-null  float64
 14  has_restroom          171996 non-null  float64
 15  buil

In [64]:
df.describe()

,cat2_slug,cat3_slug,city_slug,user_type,building_size,floor,rooms_count,total_floors_count,unit_per_floor,is_rebuilt,has_restroom,building_direction,flag,value,age,neighborhood_encoded
count,171996.000000,171996.000000,171996.000000,171996.000000,171996.000000,171996.000000,171996.000000,171996.000000,171996.000000,171996.000000,171996.000000,171996.000000,171996.000000,171996.000000,171996.000000,171996.000000
mean,0.442598,0.442598,198.119561,0.893724,0.804581,0.568497,2.007884,0.524347,0.283817,0.641050,1.609473,1.542577,0.999622,0.803285,10.705016,0.888906
std,0.496695,0.496695,71.506451,0.308191,0.098007,0.171961,0.722270,0.283866,0.267389,0.479694,0.536511,0.590737,0.746596,0.249112,7.337992,0.068403
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,0.000000,0.000000,128.000000,1.000000,0.732784,0.396265,2.000000,0.441249,0.000000,0.000000,1.000000,1.000000,0.000000,0.729531,4.000000,0.838100
50%,0.000000,0.000000,251.000000,1.000000,0.801893,0.523834,2.000000,0.441249,0.275170,1.000000,2.000000,2.000000,1.000000,0.888191,10.000000,0.868100
75%,1.000000,1.000000,256.000000,1.000000,0.874481,0.716192,2.000000,0.685569,0.500000,1.000000,2.000000,2.000000,2.000000,0.958239,16.000000,0.945716
max,1.000000,1.000000,275.000000,1.000000,1.000000,1.000000,5.000000,1.000000,1.000000,1.000000,2.000000,3.000000,2.000000,1.000000,34.000000,1.000000


In [65]:
df.to_csv('shahab_data.csv', index=False)